In [10]:
import json

from kafka import KafkaConsumer

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [11]:
config = dict(
    bootstrap_servers=['localhost:9092'],
    first_name='Michael',
    last_name='Hotaling'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['localhost:9092'],
 'first_name': 'Michael',
 'last_name': 'Hotaling',
 'client_id': 'HotalingMichael',
 'topic_prefix': 'HotalingMichael'}

Create a consumer without subscribing to any particular topic

In [12]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [14]:
general_consumer.topics()

{'HotalingMichael-accelerations',
 'HotalingMichael-joined',
 'HotalingMichael-locations',
 'HotalingMichael-simple',
 'HotalingMichael-windowed'}

In [15]:
general_consumer.unsubscribe()

Close the consumer, waiting indefinitely for any needed cleanup.

In [16]:
general_consumer.close()

In [17]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [18]:
consumer.subscription()

{'HotalingMichael-accelerations', 'HotalingMichael-locations'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [19]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Message metadata:
  Topic: HotalingMichael-locations
  Partition: 0
  Offset: 0
Message Key: 8324604c409d4a5ca67185636611a8ee
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: HotalingMichael-locations
  Partition: 0
  Offset: 1
Message Key: fe57803c5d31437aa1b3bc9331734bd6
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: HotalingMichael-locations
  Partition: 0
  Offset: 2
Message Key: 5a045017fca644d08636af9bf737674c
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: HotalingMichael-locations
  Partition: 0
  Offset: 3
Message Key: 0e711c163c7044ce8dbc5c4943a5a742
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: HotalingMichael-accelerations
  Partition: 0
  Offset: 0
Message Key: 617fd6df471f4c5e93f8a735ca250c36
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

STOPPING MESSAGE CONSUMER


Close the consumer, waiting indefinitely for any needed cleanup.

In [9]:
consumer.close()